In [ ]:
import os
from arc_prize.synth_data.html import get_html, make_prompt

puzzle_id = "103eff5b"
os.makedirs(f"html/{puzzle_id}", exist_ok=True)

print(make_prompt(puzzle_id))

# for model in ["claude"]:
#   for i in range(1):
#     response = get_html(puzzle_id, model, output_file_path=f"html/{puzzle_id}/{model}_{i}.html")
